In [40]:
# imports
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from datetime import datetime, date
import undetected_chromedriver as uc
from urllib.parse import urlencode, urljoin 
import json




In [41]:
def indeed_scrape(query = "junior software developer", pages=1, wait=5):
    home = 'https://www.indeed.com'
    base = "https://www.indeed.com/jobs?"
    params = {}
    params['q']=query
    params['sort']='date'

    titles = []
    companies = []
    links = []
    
    cities = []
    states= []
    countries=[]
    zips = []

    dates=[]
    mins=[]
    maxes=[]
    types = []
    descriptions = []

    # web driver setup
    driver = uc.Chrome()

    for page in range(0,pages):
        params['start'] = str(page*10)
        url = base + urlencode(params)
        driver.get(url)
        sleep(wait)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        
        jobs_list=soup.find('div', attrs={'id': 'mosaic-provider-jobcards'})

        # extracting data from each job card
        titles += [title.text for title in jobs_list.find_all('h2', class_='jobTitle')] 
        
        companies += [company.text for company in jobs_list.find_all('span',class_='companyName') ]
        
        page_links =[urljoin(home,link['href']) for link in jobs_list.find_all('a', class_='jcs-JobTitle') ]
        links += page_links

        for link in page_links[:2]:
            
            driver.get(link)
            sleep(5)
          
            html = driver.page_source
            cup = BeautifulSoup(html, 'html.parser')

            try:
                description = cup.find('div', id='jobDescriptionText')
                descriptions.append(description.text)
            except:
                descriptions.append('')

            try:
                job_script=cup.find('script', attrs={'type':"application/ld+json"})
                job_json=json.loads(str(job_script.contents[0]))
                
                try:
                    date_posted = str(datetime.strptime(job_json['datePosted'], '%a, %d %b %Y %H:%M:%S %Z').isoformat())
                    dates.append(date_posted)

                except:
                    dates.append(job_json['datePosted'])
                
            except:
                dates.append(None)
            
            try: 
                address = job_json['jobLocation']['address']
                cities.append(address.get('addressLocality'))
                states.append(address.get('addressRegion1'))
                zips.append(address.get('postalCode'))


            except:
                cities.append(None)
                states.append(None)
                zips.append(None)


            try:
                mins.append(int(job_json['baseSalary']['value']['minValue']))
                maxes.append(int(job_json['baseSalary']['value']['maxValue']))
                types.append(job_json['baseSalary']['value']['unitText'])
            except:
                mins.append(None)
                maxes.append(None)
                types.append(None)

    df = pd.DataFrame()
    df['Job_Title'] = titles[:2]
    df['Company'] = companies[:2]
    df['Link'] = links[:2]
    df['Description'] = descriptions
    df['Date Posted'] = dates
    df['Min Salary'] = mins
    df['Max Salary'] = maxes
    df['Salary Type'] = types

    return df
    
        

        


In [42]:
df=indeed_scrape(pages=1)

In [43]:
df

,Job_Title,Company,Link,Description,Date Posted,Min Salary,Max Salary,Salary Type
0,Junior Software Developer,Binary Logic IT.,https://www.indeed.com/company/Binary-Logic-IT...,Job Title:- Junior Java Full stack Developer\n...,2023-04-11T19:31:20Z,60000,80000,YEAR
1,Junior Software Developer,Binary Logic IT LLC,https://www.indeed.com/company/Binary-Logic-IT...,Job Title:- Junior Java Full stack Developer\n...,2023-04-11T20:23:04Z,60000,75000,YEAR


In [44]:
df.to_csv(f'indeed_{date.today()}.csv')
df.to_json(f'indeed_{date.today()}.json')